# Sources

https://github.com/thiery-lab/data-assimilation

## Model

The model is a [three-dimensional dynamical system developed as a simpified model of atmosphereic convection](https://en.wikipedia.org/wiki/Lorenz_system) by Lorenz [1] which is a classic example of chaotic non-periodic dynamics. A flow map $\varphi : \mathbb{R}_{\geq 0} \to (\mathbb{R}^3 \to \mathbb{R}^3)$ is defined such that $\boldsymbol{x}(\tau) = \varphi(\tau)(\boldsymbol{x}_0)$ where $\boldsymbol{x}(\tau), \tau > 0$ is the solution to the initial value problem
\begin{equation}
  \frac{\mathrm{d} x_0}{\mathrm{d} \tau} = \sigma (x_1 - x_0), \quad
  \frac{\mathrm{d} x_1}{\mathrm{d} \tau} = x_0 (\rho - x_2) - x_1, \quad
  \frac{\mathrm{d} x_2}{\mathrm{d} \tau} = x_0 x_1 - \beta x_2 \quad
  \text{and} \quad \boldsymbol{x}(0) = \boldsymbol{x}_0.
\end{equation}

Here we use the (standard) parameter values $\rho = 28, \sigma=10, \beta=\frac{8}{3}$ which exhibit chaotic dynamics. An implicit mid-point method is used to define an approximate flow map $\tilde{\varphi}(\Delta) \approx \varphi(\Delta)$ with time step $\Delta = 0.005$ and $S=10$ time steps per state update. The state update is $\boldsymbol{x}_t = \bigcirc_{s=1}^S \big(\tilde{\varphi}({\Delta})\big)(\boldsymbol{x}_{t-1}) + \boldsymbol{\xi}_t$ with additive noise with distribution $\boldsymbol{\xi}_t \sim \mathcal{N}(\mathbf{0}_3, 0.1^2 \mathbb{I}_3)$ and initial state distribution $\boldsymbol{x}_0 \sim \mathcal{N}(\mathbf{1}_3, 0.5^2 \mathbb{I}_3)$. The first $x_{0,t}$ component of the state is noisily observed $y_t = x_{0,t} + \epsilon_t$, with additive Gaussian noise $\epsilon_t \sim \mathcal{N}(0, 5^2)$.

References:

1. Lorenz, Edward Norton (1963). Deterministic nonperiodic flow. Journal of the Atmospheric Sciences. 20 (2): 130–141.

## Data Assimilation

In [ ]:
import numpy as np
import dapy.filters as filters
from dapy.models import Lorenz1963Model
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.style.use('seaborn-white')
plt.rcParams['figure.dpi'] = 100

In [ ]:
model_params = {
    'sigma': 10.,
    'rho': 28.,
    'beta': 8. / 3.,
    'initial_state_mean': 1.,
    'initial_state_std': 0.5,
    'state_noise_std': 0.1,
    'observation_function':  lambda x, t: x[..., 0:1],
    'observation_noise_std': 5.,
    'time_step': 0.1,
    'num_integrator_step_per_update': 5,
}


In [ ]:
model_params['time_step']=0.05
model_005 = Lorenz1963Model(**model_params)

In [ ]:
model_params['time_step']=0.01
model_001 = Lorenz1963Model(**model_params)

In [ ]:
model_params['time_step']=0.001
model_0001 = Lorenz1963Model(**model_params)

To represent the same strech of time\[0s, 50s) the indices vector has to be different for every model. Additional observations are calculated which will be later used in RC.

In [ ]:
_005_observations_count=1000
observation_time_indices_005 = np.arange(int(2.5*_005_observations_count))

In [ ]:
_001_observations_count=5000
observation_time_indices_001 = np.arange(int(2.5*_001_observations_count))

In [ ]:
_0001_observations_count=50000
observation_time_indices_0001 = np.arange(int(2.5*_0001_observations_count))

### Generate data from model

In [ ]:
seed = 1234

In [ ]:
rng = np.random.default_rng(seed)
state_sequence_005, observation_sequence_005 = model_005.sample_state_and_observation_sequences(
    rng, observation_time_indices_005)

In [ ]:
rng = np.random.default_rng(seed)
state_sequence_001, observation_sequence_001 = model_001.sample_state_and_observation_sequences(
    rng, observation_time_indices_001)

In [ ]:
rng = np.random.default_rng(seed)
state_sequence_0001, observation_sequence_0001 = model_0001.sample_state_and_observation_sequences(
    rng, observation_time_indices_0001)

### Infer state from observations

In [ ]:
def plot_results(results, observation_time_indices, state_sequence=None, 
                 plot_particles=False, plot_region=True, 
                 particle_skip=2, trace_alpha=0.5):
    fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(12, 9))
    for i, ax in zip(range(3), axes):
        ax.plot(results['state_mean_sequence'][:, i], 'g-', lw=1, label='Est. mean')
        if plot_region:
            ax.fill_between(
                observation_time_indices,
                results['state_mean_sequence'][:, i] - 3 * results['state_std_sequence'][:, i],
                results['state_mean_sequence'][:, i] + 3 * results['state_std_sequence'][:, i],
                alpha=0.25, color='g', label='Est. mean ± 3 standard deviation'
            )
        if plot_particles:
            lines = ax.plot(
                observation_time_indices, results['state_particles_sequence'][:, ::particle_skip, i], 
                'r-', lw=0.25, alpha=trace_alpha)
            lines[0].set_label('Particles')
        if state_sequence is not None:
            ax.plot(observation_time_indices, state_sequence[:, i], 'k--', label='Truth')
        ax.set_ylabel('$x_{0}$'.format(i))
        ax.legend(loc='upper center', ncol=4)
    ax.set_xlabel('Time index $t$')
    fig.tight_layout()
    return fig, axes

In [ ]:
def plot_lorenz(data, ground_truth=None):
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={'projection': '3d'})
    ax.plot(*data.T, '-', lw=0.5)
    if ground_truth is not None:
        ax.plot(*ground_truth.T, '-', lw=0.5)
    ax.set_xlabel('$x_0$')
    ax.set_ylabel('$x_1$')
    ax.set_zlabel('$x_2$')
    fig.tight_layout()

In [ ]:
def error_metric(ground_truth, result):
    return np.mean([sum(abs(np.subtract(*k)))/(i+1) for i, k in enumerate(zip(ground_truth, result))])

### Ensemble Transform Kalman filter (deterministic square root)

Ensemble Kalman filter with deterministic matrix square root updates.

The filtering distribution at each observation time index is approximated by alternating propagating an ensemble of state particles forward through time under the model dynamics and linearly transforming the ensemble according to a Monte Carlo estimate of the Kalman filter assimilation update due to the observations at the current time index. Here a 'square-root' ensemble Kalman filter assimilation update is used, which requires that the model has Gaussian observation noise with a known covariance, but compared to the 'perturbed observation' variant avoids the additional variance associated with sampling pseudo-observations.

References:

1. Bishop, C. H. Etherton, B. J. and  Majumdar, S. J. (2001). Adaptive sampling with the ensemble transform Kalman filter. Part I: Theoretical aspects.Mon. Wea. Rev., 129, 420–436.
2. M. K. Tippett, J. L. Anderson, C. H. Bishop, T. M. Hamill, and J. S. Whitaker, Ensemble square root filters, Monthly Weather Review, 131 (2003), pp. 1485--1490.

In [ ]:
etkf = filters.EnsembleTransformKalmanFilter()

In [ ]:
# Experiment with different values for num_particle
# How does the result change?
num_particle = 100

In [ ]:
rng = np.random.default_rng(seed)
results_etkf_005 = etkf.filter(
    model_005, observation_sequence_005[:_005_observations_count], observation_time_indices_005[:_005_observations_count], 
    num_particle=num_particle, rng=rng, return_particles=True)

In [ ]:
error_metric(state_sequence_005, results_etkf_005['state_mean_sequence'])

In [ ]:
plot_lorenz(results_etkf_005['state_mean_sequence'])

In [ ]:
rng = np.random.default_rng(seed)
results_etkf_005_half = etkf.filter(
    model_005, observation_sequence_005[:int(_005_observations_count/2)], 
    observation_time_indices_005[:int(_005_observations_count/2)], 
    num_particle=num_particle, rng=rng, return_particles=True)

In [ ]:
error_metric(state_sequence_005[:int(_005_observations_count/2)], results_etkf_005_half['state_mean_sequence'])

In [ ]:
plot_lorenz(results_etkf_005_half['state_mean_sequence'])

In [ ]:
rng = np.random.default_rng(seed)
results_etkf_005_quarter = etkf.filter(
    model_005, observation_sequence_005[:int(_005_observations_count/4)], 
    observation_time_indices_005[:int(_005_observations_count/4)], 
    num_particle=num_particle, rng=rng, return_particles=True)

In [ ]:
error_metric(state_sequence_005[:int(_005_observations_count/4)], results_etkf_005_quarter['state_mean_sequence'])

In [ ]:
plot_lorenz(results_etkf_005_quarter['state_mean_sequence'])

In [ ]:
rng = np.random.default_rng(seed)
results_etkf_001 = etkf.filter(
    model_001, observation_sequence_001[:_001_observations_count], observation_time_indices_001[:_001_observations_count], 
    num_particle=num_particle, rng=rng, return_particles=True)

In [ ]:
error_metric(state_sequence_001, results_etkf_001['state_mean_sequence'])

In [ ]:
plot_lorenz(results_etkf_001['state_mean_sequence'])

In [ ]:
rng = np.random.default_rng(seed)
results_etkf_001_half = etkf.filter(
    model_001, observation_sequence_001[:int(_001_observations_count/2)], 
    observation_time_indices_001[:int(_001_observations_count/2)], 
    num_particle=num_particle, rng=rng, return_particles=True)

In [ ]:
error_metric(state_sequence_001[:int(_001_observations_count/2)], results_etkf_001_half['state_mean_sequence'])

In [ ]:
plot_lorenz(results_etkf_001_half['state_mean_sequence'])

In [ ]:
rng = np.random.default_rng(seed)
results_etkf_001_quarter = etkf.filter(
    model_001, observation_sequence_001[:int(_001_observations_count/4)], 
    observation_time_indices_001[:int(_001_observations_count/4)], 
    num_particle=num_particle, rng=rng, return_particles=True)

In [ ]:
error_metric(state_sequence_001[:int(_001_observations_count/4)], results_etkf_001_quarter['state_mean_sequence'])

In [ ]:
plot_lorenz(results_etkf_001_quarter['state_mean_sequence'])

In [ ]:
rng = np.random.default_rng(seed)
results_etkf_0001 = etkf.filter(
    model_0001, observation_sequence_0001[:_0001_observations_count], observation_time_indices_0001[:_0001_observations_count], 
    num_particle=num_particle, rng=rng, return_particles=True)

In [ ]:
error_metric(state_sequence_0001, results_etkf_0001['state_mean_sequence'])

In [ ]:
plot_lorenz(results_etkf_0001['state_mean_sequence'])

In [ ]:
rng = np.random.default_rng(seed)
results_etkf_0001_half = etkf.filter(
    model_0001, observation_sequence_0001[:int(_0001_observations_count/2)], 
    observation_time_indices_0001[:int(_0001_observations_count/2)], 
    num_particle=num_particle, rng=rng, return_particles=True)

In [ ]:
error_metric(state_sequence_0001[:int(_0001_observations_count/2)], results_etkf_0001_half['state_mean_sequence'])

In [ ]:
plot_lorenz(results_etkf_0001_half['state_mean_sequence'])

In [ ]:
rng = np.random.default_rng(seed)
results_etkf_0001_quarter = etkf.filter(
    model_0001, observation_sequence_0001[:int(_0001_observations_count/4)], 
    observation_time_indices_0001[:int(_0001_observations_count/4)], 
    num_particle=num_particle, rng=rng, return_particles=True)

In [ ]:
error_metric(state_sequence_0001[:int(_0001_observations_count/4)], results_etkf_0001_quarter['state_mean_sequence'])

In [ ]:
plot_lorenz(results_etkf_0001_quarter['state_mean_sequence'])

## Reservoir computing

Reservoir computing is a framework for computation derived from recurrent neural network theory that maps input signals into higher dimensional computational spaces through the dynamics of a fixed, non-linear system called a reservoir. After the input signal is fed into the reservoir, which is treated as a "black box," a simple readout mechanism is trained to read the state of the reservoir and map it to the desired output. The first key benefit of this framework is that training is performed only at the readout stage, as the reservoir dynamics are fixed. The second is that the computational power of naturally available systems, both classical and quantum mechanical, can be utilized to reduce the effective computational cost.

References:

1. Tanaka, Gouhei; Yamane, Toshiyuki; Héroux, Jean Benoit; Nakane, Ryosho; Kanazawa, Naoki; Takeda, Seiji; Numata, Hidetoshi; Nakano, Daiju; Hirose, Akira (2019). "Recent advances in physical reservoir computing: A review". Neural Networks. 115: 100–123. doi:10.1016/j.neunet.2019.03.005. ISSN 0893-6080. PMID 30981085.
2. Röhm, André; Lüdge, Kathy (2018-08-03). "Multiplexed networks: reservoir computing with virtual and real nodes". Journal of Physics Communications. 2 (8): 085007. Bibcode:2018JPhCo...2h5007R. doi:10.1088/2399-6528/aad56d. ISSN 2399-6528

## Echo state network

The echo state network (ESN) is a type of reservoir computer that uses a recurrent neural network with a sparsely connected hidden layer (with typically 1% connectivity). The connectivity and weights of hidden neurons are fixed and randomly assigned. The weights of output neurons can be learned so that the network can produce or reproduce specific temporal patterns. The main interest of this network is that although its behaviour is non-linear, the only weights that are modified during training are for the synapses that connect the hidden neurons to output neurons. Thus, the error function is quadratic with respect to the parameter vector and can be differentiated easily to a linear system.

In [ ]:
import sys
sys.path.append('/tmp/pyESN')
from pyESN import ESN

### Lorenz

In [ ]:
def calculate_predictions(esn, ground_truth, learning_length): 
    pred_training = esn.fit(np.ones(learning_length), ground_truth[:learning_length])
    
    prediction_length=int(learning_length/50)
    prediction=esn.predict(np.ones(prediction_length))
    prediction_ground_truth=ground_truth[learning_length:learning_length+prediction_length] 
    
    plot_lorenz(prediction, prediction_ground_truth)

In [ ]:
#TODO: Set the number of output layers
# Experiment with different values for n_reservoir
# How does the result change?
esn = ESN(n_inputs = 1,
          n_outputs = 3,
          n_reservoir = 1024,
          spectral_radius = 1.5,
          random_state=42)

In [ ]:
calculate_predictions(esn, state_sequence_005, _005_observations_count)

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

#TODO: Show prediction result on plot
ax.plot(prediction[:,0],prediction[:,1],prediction[:,2], lw=0.5)
# ax.plot(data[future:,0], data[future:,1], data[future:,2], lw=0.5)
ax.set_xlabel("X Axis")
ax.set_ylabel("Y Axis")
ax.set_zlabel("Z Axis")
ax.set_title("Lorenz Attractor")